In [ ]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from functions import *

In [6]:
data = read_file()
model = setModel(data,time_limit=60,mipgap=None,solver='gurobi',status=True)

     0     0  234.93605    0  220  241.24109  234.93605  2.61%     -   46s
     0     0  234.93605    0  237  241.24109  234.93605  2.61%     -   46s
     0     0  234.93605    0  255  241.24109  234.93605  2.61%     -   47s
     0     0  234.93605    0  364  241.24109  234.93605  2.61%     -   47s
     0     0  234.93605    0  287  241.24109  234.93605  2.61%     -   47s
     0     0  234.93605    0  475  241.24109  234.93605  2.61%     -   48s
     0     0  234.93605    0  259  241.24109  234.93605  2.61%     -   48s
     0     0  234.93605    0  208  241.24109  234.93605  2.61%     -   49s
     0     2  234.93605    0  202  241.24109  234.93605  2.61%     -   49s
     1     5  235.34745    1  159  241.24109  235.34745  2.44%  1630   50s
   110   156  235.53757   15  222  241.24109  235.53757  2.36%   378   55s
   357   342  235.53801   30  250  241.24109  235.53757  2.36%   372   60s

Cutting planes:
  Gomory: 20
  Cover: 371
  Implied bound: 48
  Clique: 4
  MIR: 208
  StrongCG: 32

In [8]:
from datetime import timedelta

In [10]:
def charger_(N, T, K, x):
    # Initial value extraction from model
    value = []
    for n in N:
        for t in T:
            value.append(pyo.value(sum(x[k, n, t] for k in K)))

    # Convert the list to a numpy array
    array_charger = np.array(value)

    # Reshape the array to 10 chargers and 96 timesteps
    reshaped_array = array_charger.reshape(len(N), len(T))

    # Create DataFrame with the original 15-minute intervals
    charger_columns = [f'Charger {i+1}' for i in range(len(N))]
    df_charger = pd.DataFrame(reshaped_array.T, index=T, columns=charger_columns)
    
    # Extract values from the model
    value = []
    for k in K:
        for n in N:
            for t in T:
                # Convert time step to hour:minutes format
                time_str = str(timedelta(minutes=t * 15))
                time_str = time_str[:-3]  # remove seconds part
                value.append((k, n, time_str, pyo.value(x[k, n, t])))

    # Convert the list to a DataFrame
    df = pd.DataFrame(value, columns=['Bus', 'Charger', 'Time', 'Charging_Status'])
    
    # Filter to include only charging events (Charging_Status == 1)
    df_charging = df[df['Charging_Status'] == 1]
    
    return df_charger, df_charging

In [11]:
def save(model):
    #Calculate energy
    Energy,Energy_perc = energy_bus(model.K, model.T, model.e, model.C_bat)
    #Calculate power buy
    Power = power(model.T, model.w_buy) * 4
    #Calculate the charger engaged
    Chargers_enabled, Chargers_assigned = charger_(model.N, model.T, model.K, model.x)
    #Calculate objective function value
    Obj = pd.DataFrame({'Objective Value': [model.obj()]})
    #Save data to Excel with the name of t_start
    with pd.ExcelWriter('output.xlsx') as writer:  
        Energy.to_excel(writer, sheet_name='Energy')
        Energy_perc.to_excel(writer, sheet_name='SOC')
        Power.to_excel(writer, sheet_name='Power')
        Chargers_enabled.to_excel(writer, sheet_name='Charger Enabled')
        Chargers_assigned.to_excel(writer, sheet_name='Charger Assigned')
        Obj.to_excel(writer, sheet_name='Optimal value')
    print('The outputs are saved')

In [12]:
save(model)

The outputs are saved
